<a href="https://colab.research.google.com/github/fahriyegrl/food101_classification_model_tensorflow/blob/main/food_classification_project_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
## Getting food data from TensorFlow Dataset

import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    name='food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [58]:
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=101),
})

In [59]:
class_names = ds_info.features['label'].names
print(class_names)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [60]:
train_example = ds_train.take(1)
for image, label in train_example:
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())
  print("Class name: ", class_names[label.numpy()])
  print(type(image))
  print(image.shape)

Image shape:  (512, 512, 3)
Label:  21
Class name:  chocolate_cake
<class 'tensorflow.python.framework.ops.EagerTensor'>
(512, 512, 3)


In [61]:
image

<tf.Tensor: shape=(512, 512, 3), dtype=uint8, numpy=
array([[[ 32,  14,   0],
        [ 31,  13,   0],
        [ 28,  12,   0],
        ...,
        [ 54,  34,  10],
        [ 53,  33,   9],
        [ 51,  31,   7]],

       [[ 28,  10,   0],
        [ 28,  10,   0],
        [ 27,  11,   0],
        ...,
        [ 56,  36,  12],
        [ 55,  35,  11],
        [ 54,  34,   9]],

       [[ 30,  12,   0],
        [ 31,  13,   0],
        [ 31,  15,   0],
        ...,
        [ 58,  38,  13],
        [ 58,  38,  13],
        [ 58,  38,  11]],

       ...,

       [[135,  80,   0],
        [136,  81,   1],
        [137,  82,   2],
        ...,
        [ 13,   6,   0],
        [ 13,   6,   0],
        [ 13,   6,   0]],

       [[137,  82,   2],
        [137,  82,   2],
        [137,  82,   2],
        ...,
        [ 13,   6,   0],
        [ 13,   6,   0],
        [ 13,   6,   0]],

       [[136,  81,   1],
        [135,  80,   0],
        [135,  80,   0],
        ...,
        [ 14,   7,   

In [62]:
## Preprocessing for data

def prepocess_image(image, label):
  image = tf.image.resize(image, (224, 224))
  #image = image / 255.0
  return tf.cast(image, tf.float32), label

In [63]:
image_preprocessed = prepocess_image(image, label)

image.shape, image_preprocessed[0].shape

(TensorShape([512, 512, 3]), TensorShape([224, 224, 3]))

In [64]:
## Preparing datasets into batches by using tf.data

BATCH_SIZE = 32
ds_train = ds_train.map(map_func=prepocess_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.shuffle(buffer_size=1000).batch(batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

ds_test = ds_test.map(map_func=prepocess_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)





In [65]:
ds_train, ds_test

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [66]:
train_example = ds_train.take(1)
for image, label in train_example:
  print("Image shape: ", image.shape)
  print("Label: ", label.numpy())
  #print("Class name: ", class_names[label.numpy()])
  print(type(image))

Image shape:  (32, 224, 224, 3)
Label:  [47 37 77 84 49 66 55 22 13 67 13 76 44 86 80 49 38 92  5 40 18  3 64 53
 96 33 48 51 14  0 75  8]
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [67]:
## TensorBorad callbacks

check_path = "model_checkpoint/cp.weights.h5"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=check_path,
                                                 monitor='val_accuracy',
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 verbose=0)

In [68]:
## Setting mixed precision training
import tensorflow as tf
from keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')


In [69]:
mixed_precision.global_policy()

<DTypePolicy "mixed_float16">

In [70]:
import tensorflow as tf

print(tf.__version__)

2.17.0


In [71]:
#### creating a model

from keras import mixed_precision
from keras import layers

#mixed_precision.set_global_policy('mixed_float16')
input_shape = (224, 224, 3)

base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=input_shape, name='input_layer')
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)



model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [72]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pooling_layer                        │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 101)                 │         129,381 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_float32 (Activation)         │ (None, 101)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,178,952 (15.94 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [73]:
for layer in model.layers:
  print(layer.name, layer.trainable)

input_layer True
efficientnetb0 False
pooling_layer True
dense_2 True
softmax_float32 True


In [82]:
## Fitting te model
initial_epochs = 3

#cardinality = tf.data.experimental.cardinality(ds_test).numpy()
#if cardinality == tf.data.INFINITE_CARDINALITY:
#    validation_steps = None  # Let Keras handle the validation steps for infinite datasets
#else:
#    validation_steps = int(0.15 * cardinality)

history = model.fit(ds_train,
                    epochs=initial_epochs,
                    steps_per_epoch=len(ds_train),
                    validation_data=ds_test,
                    validation_steps=int(0.15 * len(ds_test)),
                    #validation_steps=validation_steps,
                    callbacks=[cp_callback])

Epoch 1/3
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 92s 31ms/step - accuracy: 0.7018 - loss: 1.1474 - val_accuracy: 0.7095 - val_loss: 1.1138
Epoch 2/3
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 2s 967us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6997 - val_loss: 1.1080
Epoch 3/3
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 66s 27ms/step - accuracy: 0.7114 - loss: 1.1208 - val_accuracy: 0.6957 - val_loss: 1.1019


In [83]:
## evaluate the model
#squeezed_ds_test = tf.squeeze(image, axis=0)

result_feature_extraction = model.evaluate(ds_test)
result_feature_extraction

790/790 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.7019 - loss: 1.1051


[1.0987850427627563, 0.7052277326583862]

In [97]:
#model.save("model_checkpoint/cp.weights.h5")
import keras

model.save('model.keras')

In [98]:
model_loaded = keras.models.load_model('model.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 8 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 4 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [100]:
model_loaded.evaluate(ds_test)

790/790 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.6988 - loss: 1.1104


[1.1054044961929321, 0.7029703259468079]

In [101]:
## Fine tuning the model

model_loaded.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pooling_layer                        │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 101)                 │         129,381 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax_float32 (Activation)         │ (None, 101)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,308,339 (16.44 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 129,387 (505.43 KB)

In [109]:
for layer in model_loaded.layers:
  layer.trainable = True
  print(layer.name, layer.trainable,  layer.dtype, layer.dtype_policy)

input_layer True float32 <DTypePolicy "mixed_float16">
efficientnetb0 True float32 <DTypePolicy "mixed_float16">
pooling_layer True float32 <DTypePolicy "mixed_float16">
dense_2 True float32 <DTypePolicy "mixed_float16">
softmax_float32 True float32 <DTypePolicy "float32">


In [121]:
for layer in model_loaded.layers[1].layers[:10]:
  layer.trainable = True
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_layer_2 True float32 <DTypePolicy "mixed_float16">
rescaling True float32 <DTypePolicy "mixed_float16">
normalization True float32 <DTypePolicy "mixed_float16">
rescaling_1 True float32 <DTypePolicy "mixed_float16">
stem_conv_pad True float32 <DTypePolicy "mixed_float16">
stem_conv True float32 <DTypePolicy "mixed_float16">
stem_bn True float32 <DTypePolicy "mixed_float16">
stem_activation True float32 <DTypePolicy "mixed_float16">
block1a_dwconv True float32 <DTypePolicy "mixed_float16">
block1a_bn True float32 <DTypePolicy "mixed_float16">


In [122]:
checkpoint_path = "fine_tune_checkpoints.keras"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [123]:
model_loaded.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["accuracy"])

In [124]:
for layer_num, layer in enumerate(model_loaded.layers[1].layers[:10]):
  print(layer_num, layer.name, layer.trainable)

0 input_layer_2 True
1 rescaling True
2 normalization True
3 rescaling_1 True
4 stem_conv_pad True
5 stem_conv True
6 stem_bn True
7 stem_activation True
8 block1a_dwconv True
9 block1a_bn True


In [125]:
# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)

In [127]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=3) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints.keras"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [129]:

history_fine_tuning = model_loaded.fit(ds_train,
                                epochs=50,
                                steps_per_epoch=len(ds_train),
                                validation_data=ds_test,
                                validation_steps=int(0.15 * len(ds_train)),
                                callbacks=[model_checkpoint, reduce_lr, early_stopping])


Epoch 1/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 310s 97ms/step - accuracy: 0.6194 - loss: 1.4987 - val_accuracy: 0.7915 - val_loss: 0.7445 - learning_rate: 1.0000e-04
Epoch 2/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7963 - val_loss: 0.7349 - learning_rate: 1.0000e-04
Epoch 3/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 185s 78ms/step - accuracy: 0.7883 - loss: 0.7849 - val_accuracy: 0.7992 - val_loss: 0.7012 - learning_rate: 1.0000e-04
Epoch 4/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8069 - val_loss: 0.6753 - learning_rate: 1.0000e-04
Epoch 5/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 188s 79ms/step - accuracy: 0.8542 - loss: 0.5443 - val_accuracy: 0.8246 - val_loss: 0.6439 - learning_rate: 1.0000e-04
Epoch 6/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 2s 685us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8126 - val_loss: 0.6636 - learning_rate: 1.0000e-04
Epoch 7/50
2367/2

In [130]:
model_loaded.save("model_fine_tune.keras")

In [137]:
!wget https://github.com/fahriyegrl/model_deployment_Food101_effnetb2/blob/0c19059628140a7bd09b908c809e4a74eeab3ba2/images/IMG_1804.jpeg

--2024-08-05 19:22:19--  https://github.com/fahriyegrl/model_deployment_Food101_effnetb2/blob/0c19059628140a7bd09b908c809e4a74eeab3ba2/images/IMG_1804.jpeg
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘IMG_1804.jpeg’

IMG_1804.jpeg           [ <=>                ] 264.37K  --.-KB/s    in 0.01s   

2024-08-05 19:22:20 (23.3 MB/s) - ‘IMG_1804.jpeg’ saved [270719]



In [171]:
import tensorflow as tf

def load_and_prep_image(filename, img_shape=224):
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  #img = img/255.
  return img



In [172]:
!wget https://www.pixelstalk.net/wp-content/uploads/2016/08/Free-Food-Images-Download.jpg

--2024-08-05 19:34:54--  https://www.pixelstalk.net/wp-content/uploads/2016/08/Free-Food-Images-Download.jpg
Resolving www.pixelstalk.net (www.pixelstalk.net)... 172.67.210.245, 104.21.61.136, 2606:4700:3032::6815:3d88, ...
Connecting to www.pixelstalk.net (www.pixelstalk.net)|172.67.210.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 1031197 (1007K) [image/jpeg]
Saving to: ‘Free-Food-Images-Download.jpg.1’

Free-Food-Images-Do 100%[===================>]   1007K  --.-KB/s    in 0.007s  

2024-08-05 19:34:55 (134 MB/s) - ‘Free-Food-Images-Download.jpg.1’ saved [1031197/1031197]



In [173]:
!wget https://images7.alphacoders.com/389/389345.jpg

--2024-08-05 19:34:57--  https://images7.alphacoders.com/389/389345.jpg
Resolving images7.alphacoders.com (images7.alphacoders.com)... 104.20.75.132, 172.67.48.187, 104.20.76.132, ...
Connecting to images7.alphacoders.com (images7.alphacoders.com)|104.20.75.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 582330 (569K) [image/jpeg]
Saving to: ‘389345.jpg.1’

389345.jpg.1        100%[===================>] 568.68K  --.-KB/s    in 0.008s  

2024-08-05 19:34:57 (71.2 MB/s) - ‘389345.jpg.1’ saved [582330/582330]



In [174]:
# Load and preprocess the image
image_tensor = load_and_prep_image("/content/389345.jpg")





In [175]:
# Add an extra axis to represent batch size (as the model expects a batch of images)
image_tensor = tf.expand_dims(image_tensor, axis=0)

In [176]:
pred = model_loaded.predict(image_tensor)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[1.1240974e-03, 1.7242560e-09, 1.3888091e-06, 1.8722211e-07,
        2.7669631e-07, 3.1599711e-07, 8.6892964e-05, 2.0201021e-08,
        1.0114272e-04, 7.1211053e-07, 3.4167437e-07, 8.9131085e-08,
        9.7714928e-07, 1.0361147e-06, 1.8371056e-03, 8.0692695e-07,
        1.2652804e-02, 3.2417161e-06, 2.3209330e-07, 1.8795487e-07,
        4.9608921e-08, 7.5985998e-04, 9.6823117e-03, 1.0202036e-05,
        2.7543078e-05, 1.0083178e-05, 1.1670382e-07, 5.3150486e-04,
        6.6896592e-07, 4.2133979e-03, 6.2242107e-06, 1.6578191e-04,
        4.7166691e-06, 1.1248599e-05, 4.9623732e-06, 2.6561670e-06,
        1.7419570e-06, 5.4060738e-08, 4.4817924e-08, 2.7116062e-05,
        2.9747659e-06, 2.5954118e-04, 4.1590043e-05, 3.1053739e-06,
        9.1244765e-08, 9.3802173e-06, 4.0745806e-05, 5.4484744e-08,
        6.5464924e-08, 6.5622284e-05, 1.3590866e-07, 5.1291397e-07,
        2.2545963e-05, 1.7697009e-04, 1.3200444e-06, 1.1876860e-04,
        2.9454171e-07, 8.4083722e-06, 3.1706624e

In [177]:
tf.argmax(pred, axis=1)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([72])>

In [179]:
label = class_names[72]
label

'pancakes'